### Hyperparameter Tuning for DeepCT-Query Reduction

In [2]:
from tira.third_party_integrations import ensure_pyterrier_is_loaded
ensure_pyterrier_is_loaded()

import pyterrier as pt
import os
import sys
import pandas as pd
from tqdm import tqdm
sys.path.append('deepct-query-reduction')
from deepct_query_reduction import DeepCTQueryReduction


index = pt.IndexRef.of(os.path.abspath('pyterrier-index/index/data.properties'))

In [3]:
MODELS = ["/deepct-models/deepct-main-01/output/model.ckpt-20000",
          "/deepct-models/deepct-main-01/output/model.ckpt-22503", 
          #"/deepct-models/deepct-title-01/output/model.ckpt-20000",
          #"/deepct-models/deepct-title-01/output/model.ckpt-21166", 
          #"/deepct-models/deepct-main-01/output/model.ckpt-0",
          #"/deepct-models/deepct-main-02/output/model.ckpt-0",
          #"/deepct-models/deepct-title-01/output/model.ckpt-0",
]
OMIT_DUPLICATES = ['True', 'False']
PROPORTIONS = ['0.05', '0.10', '0.15', '0.20', '0.25', '0.30', '0.35', '0.40', '0.45', '0.50', '0.55', '0.60', '0.65', '0.70', '0.75', '0.80', '0.85', '0.90', '0.95', '1.00']
pipelines = []

for model in MODELS:
    for omit in OMIT_DUPLICATES:
        for proportion in PROPORTIONS:
            pipelines += [model + ';' + proportion + ';' + omit]

print(len(pipelines), 'Pipelines')

80 Pipelines


In [4]:
def report_effectiveness(pipeline, dataset):
    queries = pt.io.read_topics(f'tot-{dataset}/queries.xml', 'trecxml')
    deep_ct_query_reduction = DeepCTQueryReduction(f'deepct-query-reduction/tot-{dataset}-deepct-predictions/predictions.json', pipeline).as_transformer()

    bm25 = pt.BatchRetrieve(index, wmodel="BM25")

    retrieval_pipeline = deep_ct_query_reduction >> bm25

    ret = pt.Experiment([retrieval_pipeline], queries, pt.io.read_qrels(f'tot-{dataset}/qrels.txt'), eval_metrics=["ndcg_cut_10", 'recip_rank', 'P_10', 'recall_100', 'recall_1000'])
    
    return {i: ret.iloc[0][i] for i in ['ndcg_cut_10', 'recip_rank', 'P_10', 'recall_100', 'recall_1000']}

def report_pipeline(pipeline):
    ret = {'Pipeline': pipeline}
    for dataset in ['train', 'dev']:
        for k,v in report_effectiveness(pipeline, dataset).items():
            ret[dataset + '_' + k] = v
    return ret

df = []
for pipeline in tqdm(pipelines):
    df += [report_pipeline(pipeline)]

df = pd.DataFrame(df)
df.to_json('hyperparameter-tuning-deepct-results-variant-01.jsonl', lines=True, orient='records')

  0%|          | 0/80 [00:00<?, ?it/s]

/opt/conda/lib/python3.7/site-packages/pyterrier/batchretrieve.py:307: UserWarning: Skipping empty query for qid 157
  warn("Skipping empty query for qid %s" % qid)
100%|██████████| 80/80 [46:31<00:00, 34.90s/it] 


In [11]:
df = pd.read_json('hyperparameter-tuning-deepct-results-variant-01.jsonl', lines=True)
df['Model'] = df['Pipeline'].apply(lambda i: i.split('models/deepct-')[1].split(';')[0])
df['Proportion'] = df['Pipeline'].apply(lambda i: i.split(';')[1])
df['Unique'] = df['Pipeline'].apply(lambda i: i.split(';')[2])
df[['Model', 'Proportion', 'Unique', 'train_ndcg_cut_10', 'dev_ndcg_cut_10', 'train_recall_100', 'dev_recall_100', 'train_recall_1000', 'dev_recall_1000', ]].sort_values('train_ndcg_cut_10', ascending=False).head(50)

,Model,Proportion,Unique,train_ndcg_cut_10,dev_ndcg_cut_10,train_recall_100,dev_recall_100,train_recall_1000,dev_recall_1000
10,main-01/output/model.ckpt-20000,0.55,True,0.099662,0.098784,0.286667,0.286667,0.506667,0.493333
53,main-01/output/model.ckpt-22503,0.70,True,0.098709,0.095243,0.293333,0.273333,0.513333,0.493333
12,main-01/output/model.ckpt-20000,0.65,True,0.098247,0.095243,0.293333,0.280000,0.506667,0.493333
13,main-01/output/model.ckpt-20000,0.70,True,0.098247,0.095243,0.293333,0.273333,0.513333,0.493333
50,main-01/output/model.ckpt-22503,0.55,True,0.097655,0.099538,0.293333,0.286667,0.506667,0.493333
51,main-01/output/model.ckpt-22503,0.60,True,0.097655,0.095459,0.293333,0.273333,0.513333,0.493333
11,main-01/output/model.ckpt-20000,0.60,True,0.097655,0.096108,0.293333,0.273333,0.513333,0.493333
52,main-01/output/model.ckpt-22503,0.65,True,0.096782,0.095536,0.293333,0.280000,0.506667,0.493333
54,main-01/output/model.ckpt-22503,0.75,True,0.096249,0.095243,0.293333,0.273333,0.513333,0.493333
18,main-01/output/model.ckpt-20000,0.95,True,0.095325,0.092482,0.273333,0.266667,0.500000,0.486667


In [7]:
df = pd.read_json('hyperparameter-tuning-deepct-results.jsonl', lines=True)
df['Model'] = df['Pipeline'].apply(lambda i: i.split('models/deepct-')[1])
df['Proportion'] = df['Pipeline'].apply(lambda i: i.split(';')[1])
df['Unique'] = df['Pipeline'].apply(lambda i: i.split(';')[2])
df[['Model', 'Proportion', 'Unique', 'train_ndcg_cut_10', 'train_recip_rank', 'train_P_10', 'dev_ndcg_cut_10', 'dev_recip_rank', 'dev_P_10']].sort_values('train_ndcg_cut_10', ascending=False).head(50)

,Model,Proportion,Unique,train_ndcg_cut_10,train_recip_rank,train_P_10,dev_ndcg_cut_10,dev_recip_rank,dev_P_10
10,main-01/output/model.ckpt-20000;0.55;True,0.55,True,0.099662,0.093855,0.013333,0.098784,0.096513,0.012667
53,main-01/output/model.ckpt-22503;0.70;True,0.70,True,0.098709,0.092952,0.013333,0.095243,0.091492,0.012667
12,main-01/output/model.ckpt-20000;0.65;True,0.65,True,0.098247,0.092568,0.013333,0.095243,0.091522,0.012667
13,main-01/output/model.ckpt-20000;0.70;True,0.70,True,0.098247,0.092444,0.013333,0.095243,0.091503,0.012667
178,main-01/output/model.ckpt-0;0.95;True,0.95,True,0.097899,0.092864,0.013333,0.090815,0.086585,0.012000
133,title-01/output/model.ckpt-21166;0.70;True,0.70,True,0.097794,0.089297,0.014000,0.094792,0.090741,0.012667
135,title-01/output/model.ckpt-21166;0.80;True,0.80,True,0.097794,0.089268,0.014000,0.094424,0.090336,0.012667
92,title-01/output/model.ckpt-20000;0.65;True,0.65,True,0.097794,0.089328,0.014000,0.094511,0.090084,0.012667
93,title-01/output/model.ckpt-20000;0.70;True,0.70,True,0.097794,0.089300,0.014000,0.095254,0.091297,0.012667
94,title-01/output/model.ckpt-20000;0.75;True,0.75,True,0.097794,0.089282,0.014000,0.094424,0.090325,0.012667
